In [1]:
import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np
from statsmodels.tsa.api import SimpleExpSmoothing
# import warnings
# warnings.filterwarnings('ignore' )

In [2]:

tickers = ["CROMPTON.NS","GLAND.NS"]
start = '2021-01-01'
end = '2022-04-28'
source = 'yahoo'
data = yf.download(tickers,start=start,end=end)

[*********************100%***********************]  2 of 2 completed


In [3]:
data

Adj Close                    Close                     High  \
           CROMPTON.NS     GLAND.NS CROMPTON.NS     GLAND.NS CROMPTON.NS   
Date                                                                       
2021-01-01  373.820160  2369.699951  378.049988  2369.699951  385.899994   
2021-01-04  369.518860  2447.699951  373.700012  2447.699951  380.450012   
2021-01-05  373.128021  2468.550049  377.350006  2468.550049  379.899994   
2021-01-06  379.604706  2408.449951  383.899994  2408.449951  385.350006   
2021-01-07  390.629974  2401.100098  395.049988  2401.100098  402.000000   
...                ...          ...         ...          ...         ...   
2022-04-21  380.799988  3448.699951  380.799988  3448.699951  386.899994   
2022-04-22  379.250000  3339.300049  379.250000  3339.300049  384.250000   
2022-04-25  371.549988  3293.600098  371.549988  3293.600098  378.000000   
2022-04-26  374.149994  3142.949951  374.149994  3142.949951  376.450012   
2022-04-27  374.149994  3146.050049  374.149994  3146.050049  375.450012   

                                Low                     Open               \
               GLAND.NS CROMPTON.NS     GLAND.NS CROMPTON.NS     GLAND.NS   
Date                                                                        
2021-01-01  2388.000000  373.250000  2344.899902  382.000000  2344.899902   
2021-01-04  2488.399902  369.000000  2375.000000  380.000000  2390.050049   
2021-01-05  2482.399902  367.049988  2422.949951  375.149994  2444.000000   
2021-01-06  2486.949951  373.100006  2400.149902  374.399994  2480.000000   
2021-01-07  2447.000000  385.500000  2390.050049  388.000000  2429.949951   
...                 ...         ...          ...         ...          ...   
2022-04-21  3559.399902  379.500000  3211.250000  380.000000  3211.250000   
2022-04-22  3429.850098  375.450012  3319.000000  381.000000  3429.850098   
2022-04-25  3370.000000  365.600006  3248.800049  378.000000  3325.000000   
2022-04-26  3309.850098  371.600006  3117.000000  375.000000  3300.000000   
2022-04-27  3183.149902  368.200012  3039.000000  374.000000  3139.800049   

                Volume           
           CROMPTON.NS GLAND.NS  
Date                             
2021-01-01      530682    92306  
2021-01-04     1018281   629065  
2021-01-05      824289   335971  
2021-01-06      984770   238280  
2021-01-07     1742941   137885  
...                ...      ...  
2022-04-21     3073721  1531010  
2022-04-22     1320813   333630  
2022-04-25     1252431   144373  
2022-04-26     1642636   480051  
2022-04-27      632259   353500  

[326 rows x 12 columns]

In [4]:
data1= data['Close'] # applying formulae on close column

In [5]:
data1

,CROMPTON.NS,GLAND.NS
Date,,
2021-01-01,378.049988,2369.699951
2021-01-04,373.700012,2447.699951
2021-01-05,377.350006,2468.550049
2021-01-06,383.899994,2408.449951
2021-01-07,395.049988,2401.100098
...,...,...
2022-04-21,380.799988,3448.699951
2022-04-22,379.250000,3339.300049
2022-04-25,371.549988,3293.600098


In [6]:
data1 = pd.DataFrame(data1)# converting data1 in pandas data frame\
data1.columns

Index(['CROMPTON.NS', 'GLAND.NS'], dtype='object')

In [7]:
data1.isnull().sum()

CROMPTON.NS    0
GLAND.NS       0
dtype: int64

### Formula : Moving Average

In [8]:
def RSI_dataset_function(dataset,n2=14):
    for i in dataset:
        a = dataset[i]
        change = a.diff().dropna()
        change
        change_up = change.clip(lower=0)
        change_down = (-1 * change.clip(upper=0)).abs()
        change_up,change_down
        avg_up = change_up.rolling(n2).mean()
        avg_down = change_down.rolling(n2).mean().abs()
        rs = avg_up/avg_down   # calculating rs
        rsi = 100-(100/(1+rs))  # calculating RSI
        dataset[i] = rsi
    return dataset.dropna()
RSI_dataset = RSI_dataset_function(data1)
RSI_dataset

### Formula : Exponential Moving Average

In [9]:
def relative_strength_index(df,n =14):        #created a user defined function
        
    ## percentage change
    change = df.diff().dropna()
    change.append(change)
#         return change
    df = df[1:]



    ## gain and loss
    gain = change.clip(lower=0)
    loss = change.clip(upper=0).abs()
#         return gain
#         return loss

    ## average gain and average loss
    avg_gain = gain.ewm(n, adjust=True, min_periods = n).mean()
    avg_loss = abs(loss.ewm(n, adjust=True, min_periods = n).mean())

#     avg_gain = gain.rolling(n).mean()
#     avg_loss = abs(loss.rolling(n).mean())


#         return avg_gain

    ## Relative Strength
    rs = avg_gain / avg_loss

    ## Relative Strength Index
    rsi = 100 - (100/ (1 + rs))
    return rsi.dropna(),avg_gain,avg_loss,df,gain,loss,rs

rsi,avg_gain,avg_loss,df,gain,loss,rs = relative_strength_index(data1)

In [10]:
avg_gain[10:]

,CROMPTON.NS,GLAND.NS
Date,,
2021-01-18,NaN,NaN
2021-01-19,NaN,NaN
2021-01-20,NaN,NaN
2021-01-21,7.314681,17.577137
2021-01-22,6.558332,24.021408
...,...,...
2022-04-21,2.133601,28.349397
2022-04-22,1.991361,26.459437
2022-04-25,1.858603,24.695475


In [11]:
rsi

,CROMPTON.NS,GLAND.NS
Date,,
2021-01-21,77.308837,35.088304
2021-01-22,72.410418,45.173416
2021-01-25,59.673238,42.226266
2021-01-27,65.551706,37.108160
2021-01-28,63.511323,34.295117
...,...,...
2022-04-21,48.355351,62.002084
2022-04-22,47.171728,52.952336
2022-04-25,41.734357,49.705245


In [12]:
gain[10:]

,CROMPTON.NS,GLAND.NS
Date,,
2021-01-18,0.000000,0.000000
2021-01-19,15.549988,0.000000
2021-01-20,10.700012,0.000000
2021-01-21,29.299988,0.000000
2021-01-22,0.000000,79.899902
...,...,...
2022-04-21,3.500000,166.149902
2022-04-22,0.000000,0.000000
2022-04-25,0.000000,0.000000


In [25]:
data2 = data1['CROMPTON.NS']

In [35]:
change = data2.diff()
change

Date
2021-01-01          NaN
2021-01-04    -4.349976
2021-01-05     3.649994
2021-01-06     6.549988
2021-01-07    11.149994
                ...    
2022-04-21     3.500000
2022-04-22    -1.549988
2022-04-25    -7.700012
2022-04-26     2.600006
2022-04-27     0.000000
Name: CROMPTON.NS, Length: 326, dtype: float64

Date
2021-01-01         NaN
2021-01-04   -0.011506
2021-01-05    0.009767
2021-01-06    0.017358
2021-01-07    0.029044
                ...   
2022-04-21    0.009276
2022-04-22   -0.004070
2022-04-25   -0.020303
2022-04-26    0.006998
2022-04-27    0.000000
Name: CROMPTON.NS, Length: 326, dtype: float64

In [32]:
gain =  change.clip(lower=0)
gain

Date
2021-01-01          NaN
2021-01-04     0.000000
2021-01-05     3.649994
2021-01-06     6.549988
2021-01-07    11.149994
                ...    
2022-04-21     3.500000
2022-04-22     0.000000
2022-04-25     0.000000
2022-04-26     2.600006
2022-04-27     0.000000
Name: CROMPTON.NS, Length: 326, dtype: float64

In [34]:
loss = loss = change.clip(upper=0).abs()
loss

Date
2021-01-01         NaN
2021-01-04    4.349976
2021-01-05    0.000000
2021-01-06    0.000000
2021-01-07    0.000000
                ...   
2022-04-21    0.000000
2022-04-22    1.549988
2022-04-25    7.700012
2022-04-26    0.000000
2022-04-27    0.000000
Name: CROMPTON.NS, Length: 326, dtype: float64

In [41]:
avg_gain = gain.ewm(14, adjust=True, min_periods = 14).mean()


Date
2021-01-15         NaN
2021-01-18         NaN
2021-01-19         NaN
2021-01-20         NaN
2021-01-21    7.314681
                ...   
2022-04-21    2.133601
2022-04-22    1.991361
2022-04-25    1.858603
2022-04-26    1.908030
2022-04-27    1.780828
Name: CROMPTON.NS, Length: 316, dtype: float64